In [ ]:
#plt.rcParams.keys()

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import copy
from scipy.interpolate import RegularGridInterpolator
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['font.size'] = 16

#for dynamic plotting, adapted from the interwebs
from IPython.display import clear_output
def live_plot(r,data_dict,plot_props,figsize=(7,5)):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    for label,data in data_dict.items():
        plt.plot(r,data, label=label)
    plt.title(plot_props['title'])
    plt.xlabel(plot_props['xlabel'])
    plt.ylabel(plot_props['ylabel'])
    plt.legend(frameon=False) # the plot evolves to the right
    plt.show()

Radiating Spheres via Moment Methods
==

This lab will work through evolving the time-dependent moment equations to solve for the radiation field surround a 'radiating sphere'.  This problem has an exact solution, which we will not be looking too much into, but we can compare how well the two moment schemes with different closures do at capturing the solution.  

<u>Some Defintions:</u><br>
<b>Radiating sphere:</b> A finite size region in spherical coordinates that has a non-zero emission of radiation<br>
<b>Two Moment Method:</b> A scheme for evolving the first two moments of the Boltzmann equation in time.<br>
<b>Closure:</b> An analytic, or otherwise, prescription for prescribing the $n+1$ moment in terms of the $\{0\ ...\ n\}$ moments.<br>

While this week we are discussing neutrino radiation transport, I hope it came across that it doesn't really matter whether we are talking about neutrinos or photons.  All of these methods can apply to both.  The real distinction between systems is the opacities and interactions.  Often these make the neutrino radiation transport system easier than the photon system, which means we can focuses the efforts in other areas, namely the time and spatial dependence. Here we look at a simple 1D system that we treat as time dependent but actually isn't, but it is simple. The observed time dependence will be the relaxing of the radiation field to the equilibrium value.

In [ ]:
def analytic_solution(nr,rmax,rsphere,equilibriumJ,chi):
#adapted from Ernazar Abdikamalov, solution presented in Smit et al. 1997
#essential we solve for the specific intensity as a function of radius and
#angle, then integrate the angular dependence out to get the moments.
    def calc_i(rsphere,chi,equilibriumJ,r,mu):
    
        if (r < rsphere):
            g = np.sqrt(1.0-(r/rsphere)**2*(1.0-mu**2))
            s = r*mu+rsphere*g
        else:
            xtmp = np.sqrt(1.0-(rsphere/r)**2)
            if (mu >= xtmp):
                g = np.sqrt(1.0-(r/rsphere)**2*(1.0-mu**2))
                s = 2.0*rsphere*g
            else:
              s = 0.0
        return equilibriumJ*(1.0-np.exp(-chi*s))
    
    nmu = 4000
    dmu = 2.0/nmu
    dr = rmax/nr
    
    j_mom = np.zeros(nr)
    h_mom = np.zeros(nr)
    k_mom = np.zeros(nr)
    
    for k in range(nr):
        r = (k+0.5)*dr
        for j in range(nmu):
            mu = (j+0.5)*dmu-1.0
            i_int = calc_i(rsphere,chi,equilibriumJ,r,mu)
            j_mom[k] += 0.5*i_int*dmu
            h_mom[k] += 0.5*i_int*mu*dmu
            k_mom[k] += 0.5*i_int*mu*mu*dmu

    return j_mom,h_mom,k_mom

def setup_grid(nr,ng,rmin,rmax):
    #setting up the grid
    dr = (rmax-rmin)/nr #grid spacing
    rl = np.linspace(rmin-dr*ng,rmax+dr*ng,nr+2*ng,endpoint=False) #radius of left edge of cells
    r = rl+dr*0.5 #cell center radius
    rr = rl+dr #radius of right edge of cells
    return dr,rl,r,rr
    
def declare_griddata(nr,ng):
    #declaring grid variables
    griddata = {}
    griddata['J'] = np.zeros(nr+2*ng) #energy density
    griddata['H'] = np.zeros(nr+2*ng) #momentum density 
    griddata['K'] = np.zeros(nr+2*ng) #second moment
    griddata['chi'] = np.zeros(nr+2*ng) #absorption opacity
    griddata['eta'] = np.zeros(nr+2*ng) #emissivity

    return griddata

def set_opacities(rl,r,rr,griddata,sphere_radius,sphere_optical_depth,equilibriumJ):
    sphere_chi = sphere_optical_depth/sphere_radius
    for i in range(len(r)):
        if r[i] < sphere_radius:
            #in sphere
            griddata['chi'][i] = sphere_chi
            griddata['eta'][i] = equilibriumJ*griddata['chi'][i]
    return griddata['chi'],griddata['eta']

#this is just a guess at an initial field, it will be close the higher the sphere
#optical depth is, the time dependence will evenutally sort it out
def set_initial_field(griddata,r,sphere_radius,equilibriumJ):
    for i in range(len(r)):
        if r[i] < sphere_radius:
            #in sphere
            griddata['J'][i] = equilibriumJ
            griddata['H'][i] = 0.0
        else:
            #outside sphere
            griddata['J'][i] = equilibriumJ*(sphere_radius/r[i])**2
            griddata['H'][i] = griddata['J'][i]
    return griddata['J'],griddata['H']

def solve_closure(griddata,closure,returnk=False):

    fluxfactor = np.abs(griddata['H'])/griddata['J']
    if closure=='minerbo':
        k = 1.0/3.0 + 2.0/15.0*(3*fluxfactor**2 - fluxfactor**3 + 3.0*fluxfactor**4)
        
    griddata['K'] = k*griddata['J']

    if returnk:
        return griddata['K'],k
    else:
        return griddata['K']

In [ ]:
#Just for demostraction, let's look at the initial conditions
nr=500
ng=2 #must be at least 2
rmin=0.0
rmax=1.0

sphere_radius=0.2
sphere_optical_depth=1.0
equilibriumJ = 1.0

dr,rl,r,rr = setup_grid(nr,ng,rmin,rmax)
griddata = declare_griddata(nr,ng)
griddata['chi'],griddata['eta'] = set_opacities(rl,r,rr,griddata,sphere_radius,sphere_optical_depth,equilibriumJ)
griddata['J'],griddata['H'] = set_initial_field(griddata,r,sphere_radius,equilibriumJ)
griddata['K'] = solve_closure(griddata,closure='minerbo')
griddata['h'] = griddata['H']/griddata['J']
griddata['k'] = griddata['K']/griddata['J']

#plot J,H, and K
plot_props = {'title':'Initial Data for Radiating Sphere','xlabel':'Radius','ylabel':'Moments','vars_to_plot':['J','H','K']}
vardict = {}
for var in plot_props['vars_to_plot']:
    vardict[var] = griddata[var]
live_plot(r,vardict,plot_props)

In [ ]:
#plot H/J, and K/J, the reduced moments
plot_props = {'title':'Reduced Moments','xlabel':'Radius','ylabel':'Reduced Moments','vars_to_plot':['h','k']}
vardict = {}
for var in plot_props['vars_to_plot']:
    vardict[var] = griddata[var]
live_plot(r,vardict,plot_props)

In [ ]:
#minmod limiter for TVD
def minmod(a,b):
    if a*b>0:
        if np.abs(a)<np.abs(b):
            return a
        else:
            return b
    else:
        return 0

#this is the Riemann solver
#It takes the current state of the radiation (n) and from it
#1. reconstructs the variables to the two sides of the interface
#2. determines the fluxes (see above equation) on the two sides of the interface
#3. uses the HLLE riemann solver to estimate the flux _through_ the interface
#4. returns the interface flux to the hydro solver
def get_fluxes_given_state_1D(dx,ni,ng,griddata,fluxvars,closure):

    npointsi=ni+2*ng

    if np.max(griddata['chi']*dx)>1.0: 
        print("Warning: diffusive limit")
        return 
    
    staten = {}
    staten['J'] = griddata['J']
    staten['h'] = griddata['H']/griddata['J']
    staten['H'] = griddata['H']
    staten['K'] = np.zeros(npointsi)
    
    #reconstruct to interface via TVD. Note, these are the +(p) and -(m) sides of zone i, not of the interface
    statenp = {'J':np.zeros(npointsi),
               'h':np.zeros(npointsi),
               'k':np.zeros(npointsi)}
    statenm = {'J':np.zeros(npointsi),
               'h':np.zeros(npointsi),
               'k':np.zeros(npointsi)}

    interface_fluxes = {}
    
    #first, the x direction
    for var in ['J','h']:
        #don't need these, just prevents warnings
        statenp[var][0] = staten[var][0]
        statenm[var][0] = staten[var][0]
        for i in range(1,npointsi-1):
            statenp[var][i] = staten[var][i] + 1.0/2.0*minmod(staten[var][i]-staten[var][i-1],staten[var][i+1]-staten[var][i])
            statenm[var][i] = staten[var][i] - 1.0/2.0*minmod(staten[var][i]-staten[var][i-1],staten[var][i+1]-staten[var][i])
        #don't need these, just prevents warnings
        statenp[var][-1] = staten[var][-1]
        statenm[var][-1] = staten[var][-1]
    
    statenp['H'] = statenp['h']*statenp['J']
    statenm['H'] = statenm['h']*statenm['J']
    
    statenp['K'],kp = solve_closure(statenp,closure,returnk=True)
    statenm['K'],km = solve_closure(statenm,closure,returnk=True)


    #riemann problem is interface_flux(i) = R(statenp(i),statenm(i+1))
    #HLLE riemann solver is 
    #interface_flux[var] = [FL[var]*lambdamax - FR[var]*lambdamin - 
    #                        (lambdamax*lambdamin)*(stateR[var]-stateL[var])]/(lambdamax-lambdamin)
    
    fluxesp = {}
    fluxesm = {}
    #first get fluxes at interfaces, this variable is passed so we can generalize the riemann solver
    fluxesp['J'] = statenp[fluxvars['J']]
    fluxesm['J'] = statenm[fluxvars['J']]

    fluxesp['H'] = statenp[fluxvars['H']]
    fluxesm['H'] = statenm[fluxvars['H']]

    lambdathickmin = -1.0/np.sqrt(3.0) 
    lambdathickmax = 1.0/np.sqrt(3.0) 
    lambdathinmin = -1.0
    lambdathinmax = 1.0

    lambdaminp = 3.0*(1.0-kp)/2.0*lambdathickmin + (3.0*kp-1.0)/2.0*lambdathinmin
    lambdamaxp = 3.0*(1.0-kp)/2.0*lambdathickmax + (3.0*kp-1.0)/2.0*lambdathinmax
    lambdaminm = 3.0*(1.0-km)/2.0*lambdathickmin + (3.0*km-1.0)/2.0*lambdathinmin
    lambdamaxm = 3.0*(1.0-km)/2.0*lambdathickmax + (3.0*km-1.0)/2.0*lambdathinmax
   
    interface_fluxes = {}
    for var in ['J','H']:
        interface_fluxes[var] = np.zeros(npointsi)
        for i in range(ng-1,npointsi-ng+1):
            lambdamax = np.maximum(lambdamaxp[i],lambdamaxm[i+1])
            lambdamin = np.minimum(lambdaminp[i],lambdaminm[i+1])
            interface_fluxes[var][i] = ((
                        fluxesp[var][i]*lambdamax - 
                        fluxesm[var][i+1]*lambdamin + 
                        lambdamin*lambdamax*(statenm[var][i+1]-statenp[var][i]))/
                        (lambdamax-lambdamin))

    return interface_fluxes

#this fills in the boundary condtions
def do_boundaries(r,nr,ng,griddata):

    for i in range(ng):
        #inner boundary, reflecting
        griddata['J'][i] = griddata['J'][2*ng-1-i]
        griddata['H'][i] = -griddata['H'][2*ng-1-i]
        #outerboundary, 1/r^2
        griddata['J'][ng+nr+i] = griddata['J'][ng+nr-1]*(r[ng+nr-1]/r[ng+nr+i])**2
        griddata['H'][ng+nr+i] = griddata['H'][ng+nr-1]*(r[ng+nr-1]/r[ng+nr+i])**2

    return griddata
    

In [ ]:
#this is the core routine that evolve the state forward in time one step.
def evolve_one(rl,r,rr,nr,ng,griddata,closure,Deltat):
    
    npoints = nr+2*ng
    
    statenp1 = {'J':np.zeros(npoints),'H':np.zeros(npoints)}
    Deltar = rr[0]-rl[0] #assuming uniform grid

    #riemann solver to get interface fluxes
    fluxvars = {'J':'H','H':'K'}
    interface_fluxes = get_fluxes_given_state_1D(Deltar,nr,ng,griddata,fluxvars,closure)

    #set the spatial flux terms
    spatial_flux_J = np.zeros(npoints)
    spatial_flux_J[1:] = (interface_fluxes['J'][1:]*rr[1:]**2 - interface_fluxes['J'][:-1]*rl[1:]**2)/(Deltar*r[1:]**2)

    spatial_flux_H = np.zeros(npoints)
    spatial_flux_H[1:] = (interface_fluxes['H'][1:]*rr[1:]**2 - interface_fluxes['H'][:-1]*rl[1:]**2)/(Deltar*r[1:]**2)

    #solve for n+1 time
    statenp1_J = (griddata['J'] + Deltat*(griddata['eta'] - spatial_flux_J))/(1.0+griddata['chi']*Deltat)
    statenp1_H = (griddata['H'] - Deltat*(spatial_flux_H + statenp1_J*(griddata['K']/griddata['J']-1.0)/r))/(1.0+griddata['chi']*Deltat)

    #return to driver
    return statenp1_J,statenp1_H

In [ ]:
#this evolves the radiating sphere over many time steps.  It also takes care of updating plot
#every once and a while as well as periodically saving state to return at the end.
def evolve_RS(rl,r,rr,nr,ng,C,tend,initialdata,closure='minerbo',plot_props={'doplot':False}):

    t = 0
    times = {}
    states = {}

    Deltax = rr[0]-rl[0]
    
    count=0
    outputcount = 0
    outputeveryn = 10

    griddata = copy.copy(initialdata)

    while t<tend-1e-10:
        if count%outputeveryn==0:
            times[outputcount] = t
            states[outputcount] = copy.copy(griddata)
            outputcount += 1

        if plot_props['doplot']:
            if count%plot_props['outputeveryn']==0:
                vardict = {}
                plot_props['title'] = "t={time:.2f} s".format(time = t)
                for var in plot_props['vars_to_plot']:
                    vardict[var] = griddata[var]
                live_plot(r,vardict,plot_props)

        #set time step, note, this doesn't use the sound speed, however we are modelling
        #this off of McCorquodale \& Colella, \emph{CAMCOS} 6 1 (2011), 
        #this uses \Delta t = 0.192\Delta x
        Deltat = C*Deltax
        if t+Deltat>tend: Deltat = tend-t

        #do boundaries
        griddata = do_boundaries(r,nr,ng,griddata)
        
        #evolve the density
        statenp1_J,statenp1_H = evolve_one(rl,r,rr,nr,ng,griddata,closure,Deltat)
        griddata['J'] = statenp1_J
        griddata['H'] = statenp1_H

        #do boundaries
        griddata = do_boundaries(r,nr,ng,griddata)

        #set closure
        griddata['K'] = solve_closure(griddata,closure)

        griddata['k'] = griddata['K']/griddata['J']
        griddata['h'] = griddata['H']/griddata['J']
        
        #go to the next time step
        t = t+Deltat
        count += 1

    times[outputcount] = t
    states[outputcount] = copy.copy(griddata)
    outputcount += 1

    if plot_props['doplot']:
        vardict = {}
        plot_props['title'] = "t={time:.2f} s".format(time = t)
        for var in plot_props['vars_to_plot']:
            vardict[var] = griddata[var]
        live_plot(r,vardict,plot_props)

    print("step: ",count,"time: ",t)
    
    return times,states

In [ ]:
#our evolution, to match Smit et al. 1997 test (also O'Connor 2015 test)
nr=800
ng=2 #must be at least 2
rmin=0.0
rmax=3.0
C=0.25 # our use of forward Euler limits C to ~0.25
tend = 4.0

sphere_radius=1.0
sphere_optical_depth=4
equilibriumJ = 0.8


In [ ]:
j_analytic,h_analytic,k_analytic = analytic_solution(nr,rmax,sphere_radius,equilibriumJ,sphere_optical_depth/sphere_radius)

In [ ]:
dr,rl,r,rr = setup_grid(nr,ng,rmin,rmax)
griddata = declare_griddata(nr,ng)
griddata['chi'],griddata['eta'] = set_opacities(rl,r,rr,griddata,sphere_radius,sphere_optical_depth,equilibriumJ)
griddata['J'],griddata['H'] = set_initial_field(griddata,r,sphere_radius,equilibriumJ)
griddata['K'] = solve_closure(griddata,closure='minerbo')
griddata['k'] = griddata['K']/griddata['J']
griddata['h'] = griddata['H']/griddata['J']

plot_props = {'doplot':True,'outputeveryn':10,'title':'t=','xlabel':'Radius','ylabel':'Moments','vars_to_plot':['J','h','k']}

times,states = evolve_RS(rl,r,rr,nr,ng,C,tend,griddata,plot_props=plot_props)